🧱 1️⃣ Install Dependencies

In [2]:
!pip install -q transformers datasets accelerate bitsandbytes peft sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.9 MB/s eta 0:00:00


🧠 2️⃣ Import Libraries

In [1]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
)
import torch

🧩 3️⃣ Inspect and Load the Data

In [2]:
from datasets import load_dataset

# Load your uploaded JSON file directly
dataset = load_dataset("json", data_files="/content/combined_dataset.json")

# View sample
print(dataset)
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})
{'Context': "I'm going through some things with my feelings and myself. I barely sleep and I do nothing but think about how I'm worthless and how I shouldn't be here.\n   I've never tried or contemplated suicide. I've always wanted to fix my issues, but I never get around to it.\n   How can I change my feeling of being worthless to everyone?", 'Response': "If everyone thinks you're worthless, then maybe you need to find new people to hang out with.Seriously, the social context in which a person lives is a big influence in self-esteem.Otherwise, you can go round and round trying to understand why you're not worthless, then go back to the same crowd and be knocked down again.There are many inspirational messages you can find in social media. \xa0Maybe read some of the ones which state that no person is worthless, and that everyone has a good purpose to their life.Also, since our

🧩 Prepare for Language Modeling

Combine context and response into one conversational string so the model learns counselor-style replies.

In [3]:
def format_conversation(example):
    return {
        "text": f"Client: {example['Context'].strip()}\nCounselor: {example['Response'].strip()}"
    }

dataset = dataset["train"].map(format_conversation)


🦙 6️⃣ Choose Base Llama Model

You can choose any open-source variant:

    Model	                         Parameter	   Notes<br>
    "meta-llama/Meta-Llama-3-8B"	  8B	      Best balance of quality/performance
    "meta-llama/Llama-2-7b-hf"	      7B	      Older but lightweight
    "meta-llama/Meta-Llama-3-70B"	  70B	      For multi-GPU clusters

For Colab, stick with the 8B or 7B versions.

⚙️ 8️⃣ Load Model

In [4]:
model_name = "NousResearch/Llama-2-7b-chat-hf" # Changed to a publicly available model

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Llama has no pad_token by default

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16, # Changed to float16 for less memory usage
)

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

🧾 7️⃣ Tokenize Data

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=1024)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])



⚙️ Data Collator

In [6]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

🚀 🔟 🧮 Training Configuration

In [9]:
training_args = TrainingArguments(
    output_dir="./llama3_counseling_domain",
    per_device_train_batch_size=1,  # Reduced batch size
    gradient_accumulation_steps=32, # Further increased gradient accumulation steps
    learning_rate=1e-5,
    num_train_epochs=2,
    fp16=True,
    save_strategy="epoch",
    logging_steps=50,
    report_to="none",
    gradient_checkpointing=True, # Enabled gradient checkpointing
    optim="adamw_torch", # Specify AdamW optimizer
)

🚀 7️⃣ Start Training

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 10.12 MiB is free. Process 150610 has 14.73 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 21.72 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

💾 8️⃣ Save the Fine-Tuned Model

In [ ]:
trainer.save_model("./llama3_counseling_domain")
tokenizer.save_pretrained("./llama3_counseling_domain")


🧪 9️⃣ Test Generation

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="./llama3_counseling_domain", tokenizer=tokenizer)

prompt = "Client: I feel anxious and worthless lately. What should I do?\nCounselor:"
print(pipe(prompt, max_new_tokens=100)[0]["generated_text"])
